this report will check the time-varying response of certain models in `scripts/training/yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/submit_20200218.py`

I will examine some high-performing models explored in `basic_for_20200218_cc2normed.ipynb`.

Will examine time-varying responses  on 1) training set 2) testing set. I believe results should be similar.



will examine those CLS=4 models

according to `basic_for_20200218_cc2normed.ipynb`, I will check those that appear in `corr_mean_kernel3` plus two layer models (`corr_mean_kernel3` only contains 3-layer models), along with some other restrictions



In [1]:
from sys import path
from os.path import relpath, realpath, abspath, join, exists
from itertools import product

from functools import partial

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

pd.set_option('display.max_rows', 999)

In [2]:
from torchnetjson.builder import build_net

In [3]:
from thesis_v2 import dir_dict
from thesis_v2.data import load_data_helper
from thesis_v2.training.training_aux import load_training_results
from thesis_v2.training_extra.misc import count_params
from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import load_modules

load_modules()

In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200218 as param_iterator_obj,
    keygen
)

In [5]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [6]:
from thesis_v2.training_extra.data import generate_datasets
from thesis_v2.training_extra.evaluation import eval_fn_wrapper as eval_fn_wrapper_neural
from thesis_v2.training.training import eval_wrapper
from thesis_v2.data.prepared.yuanyuan_8k import get_data, get_neural_data

In [7]:
from torch.backends import cudnn
import torch
cudnn.enabled = True
cudnn.deterministic = True
cudnn.benchmark = False

_data_cache = dict()

def get_data_cached(input_size, seed):
    if (input_size, seed) not in _data_cache:
        _data_cache[input_size, seed] = get_data('a', 200, input_size,
                                                 ('042318', '043018', '051018'),
                                                 scale=0.5,
                                                 seed=seed)
    return _data_cache[input_size, seed]

In [8]:
def plot_responses(responses_3d):
    # time x num_image x num_neuron
    
    n_time, n_image, n_neuron = responses_3d.shape
    
    for idx_neuron in range(n_neuron):
        responses_3d_this = responses_3d[:,:,idx_neuron]
        responses_mean = responses_3d_this.mean(axis=1)
        responses_std = responses_3d_this.std(axis=1)
        print(idx_neuron, responses_mean, responses_std)

In [9]:
def load_everything():
    _df_collect = []
    for idx, param in enumerate(param_iterator_obj().generate()):
        if idx % 100 == 0:
            print(idx)
            
#         if idx >= 100:
#             break
            
#         print(len(param))
#         print(len(param))
        assert len(param) == 22
        
        assert param['kernel_size_l1'] == 9
        assert param['ff_1st_block']
        
        
        
        assert param['split_seed'] == 'legacy'
        assert param['pooling_ksize'] == 3
        assert param['pooling_type'] == 'avg'
        assert not param['bn_after_fc']
        assert param['rcnn_bl_psize'] == 1
        assert param['rcnn_bl_ptype'] is None
        assert param['input_size'] == 50
        
        if not (param['act_fn'] == 'relu' and param['out_channel']==16 and param['rcnn_bl_cls']==4 and param['kernel_size_l23']==3):
            continue
        
        
        # 12 to go
        # remove smoothness_name and scale_name, 10 to go
        
#         assert param['model_seed'] == 0
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
        result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        num_epochs = [len(x) for x in result['stats_all']]
        
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        
        
        print(param,  ((cc_raw/cc_max_all_neurons)**2).mean())
        
        
        # ok. time to load data and get response.
        
        datasets = get_data_cached(param['input_size'], param['split_seed'])


        datasets = {
            'X_train': datasets[0].astype(np.float32),
            'y_train': datasets[1],
            'X_val': datasets[2].astype(np.float32),
            'y_val': datasets[3],
            'X_test': datasets[4].astype(np.float32),
            'y_test': datasets[5],
        }

        # only the test one is needed.
        datasets_all = generate_datasets(
            **datasets,
            per_epoch_train=True, shuffle_train=True,
        )
        
        datasets_test = datasets_all['test']
        datasets_train = datasets_all['train']

        result_on_the_go = eval_wrapper(result['model'].cuda(),
                                        datasets_test,
                                        'cuda',
                                        1,
                                        partial(eval_fn_wrapper_neural, loss_type=param['loss_type'], yhat_reduce_axis=1),
                                        (lambda dummy1,dummy2,dummy3: torch.tensor(0.0)),
                                        return_responses=True
                                       )
        
        ret = np.asarray(result_on_the_go['corr'])
        # these should be similar.
        print(result_on_the_go['corr_mean'], result['stats_best']['stats']['test']['corr_mean'], ret.mean())
        
        
        responses_test = np.concatenate([x[0] for x in result_on_the_go['responses']], axis=1)
        print(responses_test.shape)
        plot_responses(responses_test)
        
        

In [10]:
df = load_everything()

# no mean value change across time, which is expected given the loss function.

# seems that std value increases across time.
# maybe higher std <-> more diverity and higher accuracy in prediction?

0
OrderedDict([('split_seed', 'legacy'), ('model_seed', 0), ('act_fn', 'relu'), ('loss_type', 'mse'), ('input_size', 50), ('out_channel', 16), ('num_layer', 2), ('kernel_size_l1', 9), ('kernel_size_l23', 3), ('pooling_ksize', 3), ('pooling_type', 'avg'), ('bn_after_fc', False), ('scale_name', '0.01'), ('scale', '0.01'), ('smoothness_name', '0.000005'), ('smoothness', '0.000005'), ('rcnn_bl_cls', 4), ('rcnn_bl_psize', 1), ('rcnn_bl_ptype', None), ('rcnn_acc_type', 'cummean'), ('ff_1st_block', True), ('ff_1st_bn_before_act', True)]) 0.643427571224131
0.5272912804283183 0.5272912811855258 0.5272912804283183
(4, 1600, 79)
0 [0.50422865 0.5017174  0.49815133 0.49850366] [0.09411139 0.11354399 0.11820514 0.11865228]
1 [0.50053835 0.5048372  0.5018738  0.49920624] [0.1324833  0.15013885 0.15246385 0.15055148]
2 [0.49595958 0.50095403 0.50131667 0.49855056] [0.11442079 0.13320342 0.14078228 0.14100805]
3 [0.504273   0.496824   0.49580646 0.49673298] [0.0803514  0.09282885 0.09707547 0.09749792

0.5296401088019237 0.5296401088019237 0.5296401088019237
(4, 1600, 79)
0 [0.499723   0.5056543  0.5054662  0.50516313] [0.09540382 0.1186468  0.12566103 0.12745754]
1 [0.49693608 0.5026462  0.5034317  0.4998475 ] [0.13229126 0.1487624  0.15274987 0.15143989]
2 [0.50025195 0.49809387 0.49797618 0.49588805] [0.11310807 0.1390593  0.14109428 0.13876729]
3 [0.49967462 0.49535576 0.4988037  0.49912658] [0.08152286 0.09817181 0.10038929 0.09871277]
4 [0.50746596 0.5030357  0.50170213 0.5020291 ] [0.13508604 0.1664184  0.17230386 0.17028493]
5 [0.48612335 0.5060978  0.5066105  0.50197625] [0.10370734 0.11958805 0.12413372 0.123964  ]
6 [0.50737864 0.49879363 0.49485263 0.49419135] [0.15947533 0.18845578 0.19276088 0.19362973]
7 [0.4841717  0.50207895 0.5042547  0.50186336] [0.11430355 0.14567897 0.15440194 0.1551924 ]
8 [0.5080003  0.49802512 0.50008845 0.49861237] [0.19444427 0.2156708  0.21317211 0.20438926]
9 [0.50420886 0.49770245 0.49910995 0.49727795] [0.18759604 0.19937992 0.19179872 0

22 [0.5050951  0.50584936 0.50426644 0.5030123 ] [0.05727531 0.06691165 0.07114509 0.07242731]
23 [0.49688208 0.49573413 0.49536362 0.49488807] [0.06420494 0.06576633 0.06593552 0.06505529]
24 [0.4926658  0.49481857 0.49616355 0.49700966] [0.19586442 0.19743603 0.19889647 0.19614379]
25 [0.49203134 0.48986998 0.49005318 0.49030906] [0.1692801  0.17168312 0.1735145  0.17207009]
26 [0.5003083  0.49978745 0.49949065 0.49904236] [0.14722043 0.15442656 0.15682678 0.15539905]
27 [0.5155987  0.5131929  0.5113049  0.50962967] [0.16174923 0.17785014 0.18080649 0.18052746]
28 [0.49271974 0.49228868 0.49291217 0.4977492 ] [0.18359287 0.19122034 0.19116057 0.18755594]
29 [0.5041738  0.50079924 0.4990745  0.49741325] [0.1836258  0.18868019 0.18995693 0.18804538]
30 [0.49808785 0.4981035  0.49779883 0.49838462] [0.09014586 0.09784096 0.0989541  0.0976206 ]
31 [0.5002277  0.50159216 0.5012783  0.49973992] [0.09014034 0.09610929 0.09682744 0.09531257]
32 [0.49772853 0.5035588  0.5035471  0.49869895] [

37 [0.50835836 0.5097623  0.51050204 0.509315  ] [0.22259045 0.22837238 0.23077475 0.2283066 ]
38 [0.5101421  0.50849044 0.50808054 0.5067114 ] [0.1728235  0.1752051  0.17637536 0.17399818]
39 [0.49675202 0.49724388 0.49689555 0.49645153] [0.10516556 0.10918455 0.11223104 0.11130358]
40 [0.50250727 0.50443673 0.5021157  0.4994117 ] [0.163552   0.17044517 0.17722534 0.17565046]
41 [0.50542426 0.5035561  0.50318766 0.5022528 ] [0.17086463 0.17324825 0.17296727 0.17024457]
42 [0.49421918 0.49750856 0.4976718  0.49489447] [0.19068053 0.19233166 0.19332515 0.19226357]
43 [0.50369227 0.5107782  0.510687   0.5087477 ] [0.1669587  0.17595303 0.17939149 0.17626365]
44 [0.49604216 0.49836132 0.4986774  0.497035  ] [0.2949446  0.29754278 0.2992087  0.2946377 ]
45 [0.5007158  0.5035818  0.50459415 0.5043609 ] [0.12646289 0.12886237 0.12950931 0.12812671]
46 [0.48978728 0.49452668 0.49724868 0.4977421 ] [0.21740341 0.2217884  0.22494468 0.22429869]
47 [0.4797174 0.5055862 0.5074432 0.5076109] [0.17

OrderedDict([('split_seed', 'legacy'), ('model_seed', 0), ('act_fn', 'relu'), ('loss_type', 'poisson'), ('input_size', 50), ('out_channel', 16), ('num_layer', 2), ('kernel_size_l1', 9), ('kernel_size_l23', 3), ('pooling_ksize', 3), ('pooling_type', 'avg'), ('bn_after_fc', False), ('scale_name', '0.01'), ('scale', '0.01'), ('smoothness_name', '0.000005'), ('smoothness', '0.000005'), ('rcnn_bl_cls', 4), ('rcnn_bl_psize', 1), ('rcnn_bl_ptype', None), ('rcnn_acc_type', 'cummean'), ('ff_1st_block', True), ('ff_1st_bn_before_act', False)]) 0.6312801282473962
0.5220798792163259 0.5220798782066532 0.5220798792163259
(4, 1600, 79)
0 [0.49000648 0.4990873  0.49642006 0.49574426] [0.10420072 0.1229663  0.1235375  0.12242104]
1 [0.4868142  0.49751186 0.49412492 0.4893647 ] [0.12357068 0.13991204 0.14281112 0.14224854]
2 [0.50171787 0.50325906 0.50205094 0.5034095 ] [0.11497784 0.13217638 0.13484369 0.13853632]
3 [0.49529433 0.49244606 0.49766585 0.50025916] [0.07807451 0.10020684 0.10184507 0.1027

...

0.5308949523365659 0.5308949521613316 0.5308949523365659
(4, 1600, 79)
0 [0.49679527 0.50230706 0.50424767 0.506779  ] [0.12821536 0.1348193  0.13683039 0.13655768]
1 [0.49384788 0.49985182 0.50023633 0.49913538] [0.14522359 0.15068011 0.15211366 0.150859  ]
2 [0.49329686 0.4978398  0.49849716 0.4982121 ] [0.1323607  0.13749738 0.13854216 0.13748221]
3 [0.49968797 0.49895713 0.500218   0.50221574] [0.09277924 0.10184365 0.10390773 0.10476112]
4 [0.49520272 0.5023373  0.5054177  0.50907063] [0.16961044 0.17605115 0.1791028  0.18019588]
5 [0.5022342  0.5026355  0.50360894 0.5026998 ] [0.12272543 0.12952484 0.13024583 0.12823594]
6 [0.4918262  0.49484554 0.49404863 0.4898497 ] [0.18276829 0.19837211 0.20366813 0.2014869 ]
7 [0.4983921  0.49564636 0.49755043 0.49999237] [0.15326399 0.1590279  0.16201326 0.16217464]
8 [0.497212   0.4946142  0.49257562 0.49227303] [0.20306782 0.20602725 0.20658046 0.20423111]
9 [0.50184894 0.49706504 0.49620804 0.49792546] [0.19054995 0.19571799 0.19742298 0

13 [0.4967685  0.49647668 0.49479154 0.49452493] [0.10202736 0.10261523 0.10805415 0.10893647]
14 [0.49672464 0.49881154 0.4972368  0.49606687] [0.12242111 0.12441904 0.1253623  0.12514095]
15 [0.5032746  0.5025298  0.502963   0.50377643] [0.14619328 0.15032601 0.15145887 0.15069863]
16 [0.4937714  0.49689153 0.4970021  0.49355477] [0.38301674 0.40530768 0.41449142 0.4135735 ]
17 [0.50463015 0.50435907 0.5020335  0.50030243] [0.11635521 0.11659071 0.11722094 0.11548482]
18 [0.5026517  0.5034664  0.5038208  0.50379235] [0.10824008 0.11132903 0.11172354 0.11020146]
19 [0.5049927 0.502872  0.5030642 0.503701 ] [0.15580821 0.1576815  0.15798213 0.15566337]
20 [0.50032854 0.5007415  0.50375694 0.5042209 ] [0.09876172 0.10358717 0.10486    0.1056686 ]
21 [0.5011318  0.5018047  0.500866   0.49921668] [0.12389804 0.125664   0.1262725  0.12483934]
22 [0.4970638  0.49716184 0.49642605 0.49577513] [0.0593412  0.06512535 0.06635589 0.06651081]
23 [0.49829385 0.4967672  0.49500787 0.49405852] [0.05

70 [0.4935508  0.49871388 0.49597415 0.49280894] [0.16114452 0.19265482 0.19699241 0.19508514]
71 [0.46426955 0.48134467 0.5029815  0.49968323] [0.31952968 0.36269283 0.4115448  0.4238536 ]
72 [0.4635667  0.4996954  0.5044782  0.49930847] [0.17113808 0.24187295 0.2732838  0.28016022]
73 [0.48303586 0.4887844  0.4976349  0.49338055] [0.11584523 0.13376862 0.1495262  0.15553737]
74 [0.4923171  0.49576476 0.5005368  0.5014099 ] [0.05526508 0.06625788 0.07245761 0.07484546]
75 [0.48846725 0.5031157  0.5043457  0.49908212] [0.17578827 0.20829396 0.21009259 0.20620352]
76 [0.48604515 0.4980056  0.50295883 0.5020503 ] [0.1276901  0.16407903 0.18289116 0.19074811]
77 [0.5009935  0.49623227 0.49158448 0.4940454 ] [0.14962438 0.1649224  0.17374086 0.1700626 ]
78 [0.50776505 0.49376267 0.49478152 0.49674976] [0.07034085 0.09787321 0.10570322 0.10563743]
OrderedDict([('split_seed', 'legacy'), ('model_seed', 1), ('act_fn', 'relu'), ('loss_type', 'poisson'), ('input_size', 50), ('out_channel', 16), 

0.531241354512481 0.531241354512481 0.531241354512481
(4, 1600, 79)
0 [0.49331066 0.49817407 0.49615806 0.5021141 ] [0.1278402  0.13465026 0.13281608 0.1353772 ]
1 [0.49614906 0.49876076 0.49933556 0.49737847] [0.14160988 0.15038806 0.15407409 0.15465674]
2 [0.4934213  0.49635673 0.49350014 0.49389604] [0.13430445 0.13677539 0.13418879 0.13447984]
3 [0.4936344  0.49839127 0.49933252 0.5028009 ] [0.09947434 0.10369361 0.10805391 0.10973654]
4 [0.4988988  0.5023096  0.4996918  0.50748366] [0.16438255 0.16944887 0.16564585 0.16834384]
5 [0.49993706 0.50534034 0.50136596 0.50119203] [0.12181548 0.12623027 0.12378721 0.12376325]
6 [0.49610353 0.49878085 0.4991554  0.49331966] [0.18299998 0.19397461 0.19811906 0.19714724]
7 [0.49124566 0.49644497 0.49452072 0.49735925] [0.1461664  0.16127223 0.16066591 0.1633923 ]
8 [0.49356937 0.49431136 0.49617463 0.49631777] [0.20625673 0.2077514  0.20712423 0.2069316 ]
9 [0.49248886 0.49203244 0.49454236 0.49540192] [0.18059744 0.18878634 0.1874649  0.18

7 [0.49593452 0.49619493 0.4957157  0.49267116] [0.1551885  0.16191396 0.16422427 0.16165505]
8 [0.5058307  0.5059608  0.50465155 0.5028569 ] [0.19756667 0.20018858 0.19886369 0.19819596]
9 [0.50816286 0.5055926  0.5071353  0.5084096 ] [0.17194167 0.17612588 0.17602134 0.17754158]
10 [0.50377744 0.5027282  0.5020962  0.50114006] [0.18438318 0.18743204 0.18785733 0.18615578]
11 [0.50715685 0.50573355 0.50265974 0.5034199 ] [0.21057452 0.21147567 0.2098182  0.21157388]
12 [0.49955502 0.50126576 0.4992504  0.49814683] [0.20065854 0.20141256 0.20233907 0.20318726]
13 [0.49511892 0.49577674 0.49642426 0.49527207] [0.09772124 0.10093199 0.10336218 0.10450068]
14 [0.50002134 0.5012675  0.5015576  0.49874565] [0.12323091 0.12908694 0.12734602 0.12404482]
15 [0.5008455  0.49926507 0.4983445  0.49895653] [0.14816067 0.14910127 0.14898615 0.14929497]
16 [0.48389384 0.48480153 0.47635117 0.4763707 ] [0.32477432 0.3448897  0.34298623 0.34642538]
17 [0.5029187  0.50379735 0.5028947  0.50016725] [0.1